In [3]:
import os
import json
import numpy as np
from tqdm import tqdm_notebook

magazine_path = '/data/private/Arena/datasets/magazine.json'
metadata_path = '/data/private/Arena/datasets/metadata.json'
users_path = '/data/private/Arena/datasets/users.json'
predict_path = '/data/private/Arena/datasets/predict/'
read_path = '/data/private/Arena/datasets/read/'

In [2]:
#magazine -> keyword list
magazine_list = []
for line in open(magazine_path, 'r', encoding='utf-8'):
    magazine_list.append(json.loads(line))
metadata_list = []
for line in open(metadata_path, 'r', encoding='utf-8'):
    metadata_list.append(json.loads(line))
users_list = []
for line in open(users_path, 'r', encoding='utf-8'):
    users_list.append(json.loads(line))

In [3]:
magazine_list[0], metadata_list[0], users_list[0]

({'magazine_tag_list': ['브런치북', '육아일기', '대화법', '들려주고픈이야기'], 'id': '38842'},
 {'magazine_id': 8982,
  'user_id': '@bookdb',
  'title': '사진으로 옮기기에도 아까운, 리치필드 국립공원',
  'keyword_list': ['여행', '호주', '국립공원'],
  'display_url': 'https://brunch.co.kr/@bookdb/782',
  'sub_title': '세상 어디에도 없는 호주 Top 10',
  'reg_ts': 1474944427000,
  'article_id': 782,
  'id': '@bookdb_782'},
 {'keyword_list': [],
  'following_list': ['@perytail', '@brunch'],
  'id': '#901985d8bc4c481805c4a4f911814c4a'})

## keyword / reader / writer dict and list

In [ ]:
keyword_list = ['unk', 'pad']
for data in tqdm_notebook(magazine_list):
    for keyword in data['magazine_tag_list']:
        if keyword not in keyword_list:
            keyword_list.append(keyword)
for data in tqdm_notebook(metadata_list):
    for keyword in data['keyword_list']:
        if keyword not in keyword_list:
            keyword_list.append(keyword)
for data in tqdm_notebook(users_list):
    for keyword in data['keyword_list']:
        if keyword not in keyword_list:
            keyword_list.append(keyword)
            
keyword_dict = {}
for i, keyword in enumerate(keyword_list):
    keyword_dict[keyword] = i
    
np.save('/data/private/Arena/prepro_results/keyword_dict.npy', keyword_dict)
np.save('/data/private/Arena/prepro_results/keyword_list.npy', keyword_list)

In [60]:
id2writer = ['unk']
id2reader = ['unk']
for data in tqdm_notebook(users_list):
    id2reader.append(data['id'])
    for writer in data['following_list']:
        if writer not in id2writer:
            id2writer.append(writer)

reader2id = {}
for i, reader in enumerate(id2reader):
    reader2id[reader] = i
    
writer2id = {}
for i, writer in enumerate(id2writer):
    writer2id[writer] = i
    
np.save('/data/private/Arena/prepro_results/id2reader.npy', id2reader)
np.save('/data/private/Arena/prepro_results/reader2id.npy', reader2id)
np.save('/data/private/Arena/prepro_results/id2writer.npy', id2writer)
np.save('/data/private/Arena/prepro_results/writer2id.npy', writer2id)

In [26]:
magazine2id = {'unk':0}
id2magazine = ['unk']
for data in tqdm_notebook(magazine_list):
    id_ = int(data['id'])
    
    if id_ not in id2magazine:
        magazine2id[id_] = len(id2magazine)
        id2magazine.append(id_)
        
np.save('/data/private/Arena/prepro_results/id2magazine.npy', id2magazine)
np.save('/data/private/Arena/prepro_results/magazine2id.npy', magazine2id)

## reader / item 2 elements & item dict and list

In [129]:
reader2elem = {'unk':0}
follow_list = []
follow_maxlen = 12
keyword_maxlen = 0
for data in tqdm_notebook(users_list):
    #follow_list.append(len(data['following_list']))
    id_ = reader2id[data['id']]
    follow = [writer2id['unk']] * (follow_maxlen-len(data['following_list'])) + \
             list(map(writer2id.get, data['following_list'][-follow_maxlen:]))
    reader2elem[id_] = follow

    #reader2elem[id_] = 
    #if keyword_maxlen < len(data['keyword_list']):
    #    keyword_maxlen = len(data['keyword_list'])
    #if data['keyword_list']:
    #    keywords = [kw['keyword'].split(' ') for kw in data['keyword_list'] if kw['cnt'] > 10]
    #    keywords = list(set([a for b in keywords for a in b]))
    #    print(keywords)
    #    if keyword_maxlen < len(keywords):
    #        keyword_maxlen = len(keywords)

np.save('/data/private/Arena/prepro_results/reader2elem.npy', reader2elem)

In [56]:
keywordlen_list = []
for data in tqdm_notebook(users_list):
    keywordlen_list.append(len(data['keyword_list']))

In [67]:
np.sort(keywordlen_list)[280000], len(keywordlen_list)

(0, 310758)

In [120]:
print(np.mean(follow_list))
print(sorted(follow_list)[int(len(follow_list)*0.85)])

8.420954569150272
11


In [361]:
ts_min = 1538319600 # 20181001000000 GMT+8
ts_max = 1552575600 # 20190315000000 GMT+8
ts_gap = ts_max - ts_min

keywd_maxlen = 5
item2elem = {0:[0,0,0,0,0,0,0,0]}
item_list = ['unk']
item_dict = {'unk':0}

for data in tqdm_notebook(metadata_list):

    if item_dict.get(data['id']) == None:
        item_dict[data['id']] = len(item_list)
        item_list.append(data['id'])
        
    if data['keyword_list']:
        keywd = [keyword_dict['pad']] * (keywd_maxlen-len(data['keyword_list'])) + \
                list(map(keyword_dict.get, data['keyword_list'][::-1]))
    else:
        keywd = [keyword_dict['unk']] * keywd_maxlen
    writer = writer2id[data['user_id']]
    reg_ts = int(data['reg_ts'])/1000
    reg_ts = (reg_ts-ts_min)/ts_gap if reg_ts > ts_min else 0

    if magazine2id.get(int(data['magazine_id'])) == None:
        magazine2id[int(data['magazine_id'])] = len(id2magazine)
        id2magazine.append(int(data['magazine_id']))
    mag_id = magazine2id[int(data['magazine_id'])]
    
    item2elem[item_dict[data['id']]] = [writer] + keywd + [reg_ts, mag_id]

np.save('/data/private/Arena/prepro_results/item_dict.npy', item_dict)
np.save('/data/private/Arena/prepro_results/item_list.npy', item_list)
np.save('/data/private/Arena/prepro_results/item2elem.npy', item2elem)
np.save('/data/private/Arena/prepro_results/id2magazine.npy', id2magazine)
np.save('/data/private/Arena/prepro_results/magazine2id.npy', magazine2id)

## Valid Tensor & Writer 2 items (X)

In [22]:
import numpy as np
from tqdm import tqdm_notebook
import torch

valid_writer_keywd = [[0,0,0,0,0,0,0,0,0]]


for data in tqdm_notebook(metadata_list):
    if data['keyword_list']:
        keywd = [keyword_dict['pad']] * (keywd_maxlen-len(data['keyword_list'])) + \
                list(map(keyword_dict.get, data['keyword_list'][::-1]))
    else:
        keywd = [keyword_dict['unk']] * keywd_maxlen
    writer = writer2id[data['user_id']]
    reg_ts = int(data['reg_ts'])/1000
    reg_ts = (reg_ts-ts_min)/ts_gap if reg_ts > ts_min else 0
    if magazine2id.get(int(data['magazine_id'])) == None:
        magazine2id[int(data['magazine_id'])] = len(id2magazine)
        id2magazine.append(int(data['magazine_id']))
    mag_id = magazine2id[int(data['magazine_id'])]
    item_id = item_dict[data['id']]
    
    valid_writer_keywd.append([item_id, writer] + keywd + [reg_ts, mag_id])

valid_writer_keywd = torch.from_numpy(np.array(valid_writer_keywd))
torch.save(valid_writer_keywd, '/data/private/Arena/prepro_results/valid_writer_keywd.pkl')

In [23]:
valid_writer_keywd.size()

torch.Size([643105, 9])

In [346]:
import torch
dev_tensor = torch.load('/data/private/Arena/prepro_results/valid_writer_keywd.pkl').cuda()
print(dev_tensor.size())

torch.Size([643104, 9])


In [27]:
item_dict = np.load('/data/private/Arena/prepro_results/item_dict.npy', allow_pickle=True).item()
item_list = np.load('/data/private/Arena/prepro_results/item_list.npy')

writerid2items = {}
for data in tqdm_notebook(metadata_list):
    user_id = writer2id[data['user_id']]
    id_ = item_dict[data['id']]
    keyword = keyword_dict[data['keyword_list'][0] if data['keyword_list'] is True else '없음']
    
    if writerid2items.get(user_id) == None:
        writerid2items[user_id] = [[id_, user_id, keyword]]
    else:
        writerid2items[user_id].append([id_, user_id, keyword])
        
np.save('/data/private/Arena/prepro_results/writerid2items.npy', writerid2items)


## From 2019022 : Reader 2 Read item

In [19]:
file_list = os.listdir(read_path)
file_list.sort()

In [20]:
len(file_list)

3624

In [128]:
import time
import datetime
reader2item = {}
#file_list = [x for x in os.listdir(read_path) if x.startswith('20190222')]

for read_file in tqdm_notebook(file_list[3456:]): #2월~(2952) #2월22~(3456) #2월20~(3408)
    try:
        file = open(read_path+read_file, 'r')
        data_ = file.readlines()
    except:
        print(read_file)
        continue
        
    file_ts = time.mktime(datetime.datetime.strptime(read_file[-10:], '%Y%m%d%H').timetuple()) + 32400
    file_ts = (file_ts-ts_min)/ts_gap if file_ts > ts_min else 0
    if file_ts < 0:
        print('xx')

    for line in data_:
        tokens = line.split(' ')
        try:
            reader = reader2id[tokens[0]]
        except:
            continue
        items = [[item_dict[x], file_ts] if item_dict.get(x)!=None else [item_dict['unk'], file_ts] for x in tokens[1:-1]]
        
        if reader2item.get(reader) != None:
            reader2item[reader] = reader2item[reader] + items
        else:
            reader2item[reader] = items
            
np.save('/data/private/Arena/prepro_results/reader2item.npy', reader2item)


In [294]:
len(reader2item), len(reader2id), item_list[115634]

(294366, 310759, '@tnrud572_70')

In [287]:
userid2followid = {}
for data in users_list:
    id_ = reader2id[data['id']]
    following_list = [writer2id[x] for x in data['following_list']]
    userid2followid[id_] = following_list
    
np.save('/data/private/Arena/prepro_results/userid2followid.npy', userid2followid)

In [ ]:
max_keylen = 0
for data in tqdm_notebook(metadata_list):
    keylen = len(data['keyword_list'])
    if max_keylen < keylen:
        max_keylen = keylen
print(max_keylen)

In [50]:
#reader2item = np.load('/data/private/Arena/prepro_results/reader2item.npy', allow_pickle=True).item()
readlen_list = [len(v) for v in reader2item.values()]

In [51]:
np.median(readlen_list)

6.0

# RNN Based
Train Valid Test data (from 4 to 1 : window size = 5) and Mask 

In [248]:
for k, vs in list(reader2item.items()):
    for v in vs:        
        if v[1] < 1000000000:
            print(v, k)
            break

In [134]:
rnn_train_data = []
rnn_valid_data = []
rnn_test_data = {}
window_size = 5
for reader, items_list in reader2item.items():
    if not items_list:
        continue
    items_array = np.array(items_list)
    items, read_ts = items_array[:,0].tolist(), items_array[:,1].tolist()

    if len(items) < window_size:
        items = [item_dict['unk']] * (window_size-len(items)) + items
        read_ts = [0] * (window_size-len(read_ts)) + read_ts
        
    rnn_data = []
    for i in range(len(items)-window_size+1):
        #print(len([reader, read_ts[i+window_size-1]] + items[i:i+window_size]))
        rnn_data.append([reader, read_ts[i+window_size-1]] + items[i:i+window_size])
        
    if len(items) > window_size+4:
        rnn_train_data += rnn_data[int(len(rnn_data)*0.1):]
        rnn_valid_data += rnn_data[:int(len(rnn_data)*0.1)]
    else:
        rnn_train_data += rnn_data
    rnn_test_data[reader] = rnn_data[-1]
        
np.save('/data/private/Arena/prepro_results/rnn_train_data.npy', rnn_train_data)
np.save('/data/private/Arena/prepro_results/rnn_valid_data.npy', rnn_valid_data)
##np.save('/data/private/Arena/prepro_results/rnn_test_data.npy', rnn_test_data)

6
5
33
5
9
34
7
56
8
5
42
6
12
28
28
82
9
6
19
50
171
5
94
36
10
38
5
33
13
5
5
45
15
5
49
8
5
30
117
90
79
10
5
5
170
19
12
318
11
14
90
5
7
12
5
5
5
5
121
14
19
44
5
42
58
28
5
47
28
5
12
19
61
110
151
67
11
5
288
131
59
34
28
45
37
141
14
35
5
5
8
5
495
10
35
5
5
25
18
37
7
75
12
146
5
5
23
57
513
8
9
30
261
5
16
14
589
13
5
75
5
5
7
21
14
5
5
8
11
17
18
13
65
11
5
11
10
26
10
46
11
139
42
16
169
43
148
63
5
513
29
32
5
16
5
5
5
16
5
5
5
75
49
801
5
44
157
135
122
93
5
5
46
6
5
17
5
20
21
5
5
45
5
8
135
21
146
63
88
18
27
21
40
78
25
18
151
5
132
6
45
13
11
5
10
5
5
20
11
5
9
651
5
5
5
12
49
14
5
33
83
430
47
48
45
195
30
43
88
20
29
222
77
11
64
8
5
223
36
58
8
5
50
23
21
28
8
511
14
11
5
35
7
16
30
234
27
6
7
5
297
13
5
5
5
6
58
5
5
6
32
5
6
45
12
1092
23
32
35
50
5
5
193
89
29
6
5
5
43
49
46
41
15
258
17
274
245
47
312
10
95
5
5
5
58
181
438
5
21
14
9
5
13
18
8
6
20
90
5
5
52
6
34
5
19
15
17
5
20
5
265
107
60
9
51
94
589
59
101
14
16
13
44
13
100
5
21
219
11
42
12
16
16
66
5
180


47
14
52
76
25
24
87
22
26
17
5
11
5
13
5
12
5
5
8
11
36
19
26
135
30
299
13
20
5
107
5
5
24
5
20
192
5
10
90
11
5
6
20
6
71
46
5
75
15
58
10
15
42
5
11
15
5
62
18
9
5
256
7
84
34
13
35
6
18
8
19
40
159
33
69
310
38
5
15
14
5
10
40
13
5
5
16
5
5
7
76
9
6
5
5
5
17
5
5
10
5
33
16
14
5
27
28
5
55
21
5
5
33
37
5
172
227
7
52
64
10
88
55
207
18
6
19
26
60
76
40
17
18
11
202
59
24
70
25
5
38
197
82
53
5
5
8
64
50
5
10
18
23
12
100
28
26
44
5
9
8
44
8
5
5
5
15
5
26
5
5
5
38
11
9
41
5
52
5
10
75
16
63
9
20
5
5
26
5
5
12
16
64
13
24
5
75
5
112
11
13
66
11
5
38
66
5
137
18
14
17
10
5
29
16
52
547
20
442
5
17
103
119
6
125
7
132
21
5
16
356
8
506
6
31
6
143
13
8
5
36
9
20
17
217
18
45
272
44
6
41
5
5
7
5
18
19
25
5
25
5
86
8
5
10
91
995
32
36
476
5
45
8
14
6
10
38
539
113
35
44
13
145
5
18
53
35
19
1288
32
9
17
204
151
5
19
72
9
6
10
34
134
6
9
10
5
28
5
107
5
5
22
7
10
7
11
19
13
13
5
236
176
43
5
5
5
9
5
6
5
12
13
44
28
10
5
5
13
7
6
322
93
5
5
12
111
12
5
12
5
5
5
14
35
44
75
24
49
5
9
36
8
12

231
18
5
8
5
10
5
21
21
49
5
5
20
281
7
25
72
17
15
22
5
6
281
5
15
88
31
34
26
6
5
16
5
6
19
10
102
43
5
15
39
147
29
54
36
24
20
11
16
50
5
5
6
423
31
20
5
37
527
5
5
5
5
72
179
30
13
25
34
5
5
32
5
8
18
16
5
59
6
14
25
8
5
10
5
5
5
5
5
129
178
9
20
5
8
5
20
8
5
5
14
32
12
5
52
5
5
5
5
26
104
5
114
136
10
15
8
19
15
5
5
86
17
35
5
5
5
5
7
8
5
5
96
22
51
80
37
58
5
12
62
44
8
7
7
23
10
25
16
5
7
58
104
5
5
11
26
17
166
63
51
19
5
5
29
10
5
7
36
5
7
24
16
17
22
5
29
5
33
15
23
6
68
132
10
55
50
5
7
5
59
5
161
8
6
5
57
5
5
5
57
5
5
108
5
6
12
5
8
6
16
7
196
5
20
22
20
12
5
23
58
8
5
9
5
5
141
30
5
9
16
5
5
52
5
11
21
12
7
5
5
260
15
27
6
68
5
17
5
297
1550
6
5
13
5
724
59
14
6
19
5
47
10
15
62
5
5
5
96
71
118
5
5
20
226
20
5
5
22
118
22
248
72
16
5
5
16
51
9
5
5
20
37
16
38
35
5
6
18
35
830
25
158
5
6
31
7
50
5
18
5
5
5
15
8
92
5
5
31
62
9
5
42
9
9
5
5
240
5
84
5
50
21
5
5
5
12
5
47
620
5
8
10
5
223
19
15
21
5
28
21
6
5
30
195
5
5
119
8
41
5
58
5
26
5
5
9
5
5
5
15
5
5
6
78
5
35
5
5
5
5


38
5
7
5
6
5
33
5
41
8
8
19
17
19
99
26
41
38
346
9
128
8
50
111
5
10
5
5
10
58
55
207
5
5
30
32
5
137
36
26
5
5
5
5
76
5
189
5
46
33
20
5
5
5
7
10
88
11
5
40
6
8
94
8
5
33
12
33
16
17
99
11
5
11
5
18
7
16
68
7
40
6
14
32
5
5
31
30
148
8
58
26
6
5
5
28
5
5
5
5
5
7
78
9
6
22
30
47
12
5
6
5
5
5
27
6
126
5
5
16
9
8
68
5
5
33
115
7
5
6
11
6
21
26
5
5
5
5
5
460
20
12
5
5
34
9
23
10
5
24
9
5
142
6
34
11
32
26
26
87
8
12
5
67
8
95
19
7
5
21
17
37
8
86
6
18
5
181
11
14
87
6
6
5
5
66
5
30
5
5
5
96
5
9
5
21
5
8
6
5
5
11
5
5
22
21
6
5
5
48
6
12
61
16
5
34
5
11
64
20
5
5
18
16
9
9
6
5
14
20
8
14
72
5
33
6
14
82
54
6
38
6
19
87
121
110
6
41
17
15
14
13
5
30
6
50
50
5
5
8
5
5
16
5
10
61
5
169
5
5
60
5
102
32
5
18
5
5
25
26
5
5
5
6
5
5
944
5
6
5
5
5
5
6
5
144
10
5
8
12
5
5
5
68
5
39
11
5
5
42
5
13
8
25
5
5
20
5
40
5
13
9
6
5
62
64
54
5
17
40
7
34
5
33
49
10
6
5
5
5
25
19
5
8
11
89
148
5
190
5
27
12
8
50
5
37
32
124
12
5
297
5
32
5
308
5
99
85
17
39
7
12
5
5
69
50
7
42
5
251
15
137
25
32
5
24
23
143
1

12
5
6
24
33
26
5
5
6
7
12
46
147
5
5
5
6
67
9
160
76
16
17
9
8
6
5
80
44
5
51
5
42
5
13
8
97
6
67
5
7
11
36
6
5
7
5
48
7
13
20
8
5
7
5
8
33
7
5
21
5
5
10
33
5
6
184
570
5
7
16
48
5
5
214
26
34
5
5
5
58
20
17
35
83
15
16
14
19
5
126
125
60
27
14
5
38
5
14
13
5
14
71
5
32
86
22
20
43
43
59
12
5
5
5
54
5
9
5
17
32
5
7
8
5
12
9
10
20
9
22
7
83
5
206
22
8
67
54
5
5
5
39
160
25
11
5
7
14
9
82
23
56
6
5
6
10
5
24
5
5
11
5
5
5
40
77
73
5
28
16
133
15
14
7
8
20
5
5
6
80
5
69
12
5
5
8
5
7
5
5
124
18
254
5
5
5
53
6
32
5
5
21
15
55
5
5
15
5
107
40
31
5
14
28
5
8
180
19
36
59
9
29
5
9
29
6
5
117
5
22
149
5
20
26
22
5
55
5
5
13
5
5
12
7
5
5
5
5
5
5
5
16
8
43
5
9
6
5
87
18
247
10
5
5
5
5
21
32
5
8
6
44
7
7
61
76
12
67
5
16
14
5
5
13
36
5
5
21
19
22
5
5
5
5
5
77
158
19
38
5
20
19
9
34
13
5
6
16
27
11
5
5
5
5
11
5
10
14
36
18
5
18
5
5
63
38
38
5
5
5
30
5
22
13
20
74
10
5
6
5
65
5
10
11
5
5
8
18
5
5
24
5
18
9
5
7
5
5
7
24
38
7
5
5
5
14
15
60
251
5
204
9
5
6
5
15
5
5
8
5
32
16
5
5
5
7
47
5
31
5
28
5
9
8

5
5
59
98
34
5
5
12
28
11
7
5
15
24
6
11
14
5
5
5
5
20
7
12
14
78
69
5
6
5
11
5
51
5
5
16
21
111
48
6
6
25
5
54
49
5
5
29
5
93
110
72
22
80
22
14
5
20
5
5
27
5
9
5
10
6
7
8
5
13
5
8
5
18
10
18
121
38
6
360
5
11
5
5
16
5
22
60
106
12
21
7
7
5
5
5
5
5
5
6
177
6
5
20
24
28
5
5
5
5
38
5
5
5
5
5
62
9
5
65
11
376
5
28
1745
51
5
7
5
5
31
5
12
18
5
23
5
29
5
5
5
289
5
5
36
5
5
22
10
5
5
5
8
108
5
5
5
5
492
9
10
9
43
5
14
14
56
5
5
5
5
7
5
5
8
7
5
5
5
14
5
5
12
5
23
6
5
7
5
7
58
156
37
40
24
20
169
29
27
5
45
5
11
41
40
7
5
5
5
5
9
5
29
33
32
74
68
21
5
43
88
11
10
8
6
5
5
8
7
11
46
8
13
6
25
5
198
8
30
15
5
5
5
5
5
5
65
115
11
12
5
5
28
5
5
5
6
74
14
12
5
6
34
100
5
5
5
10
41
32
57
5
43
8
128
5
33
5
28
88
15
5
5
12
10
31
5
52
5
47
5
22
98
5
5
75
13
5
5
41
9
13
59
5
184
17
5
5
13
19
13
5
14
5
50
15
20
20
64
5
5
26
8
7
57
10
40
5
5
12
7
5
5
12
147
5
5
48
23
8
5
9
5
11
5
5
5
5
7
17
39
5
5
14
5
5
5
21
153
67
8
68
44
8
8
5
5
6
9
7
14
5
5
5
5
353
6
10
24
33
10
5
6
14
5
52
57
8
41
5
5
9
5
5
30
9
40
1

KeyboardInterrupt: 

In [131]:
len(rnn_train_data), len(rnn_valid_data)

(883984, 81271)

Test data as not a dictionary (REAL)

In [135]:
pred_dev_file = '/data/private/Arena/datasets/predict/dev.users'
pred_dev_data = open(pred_dev_file, 'r').readlines()
test_data = []
a = 0
for line in pred_dev_data:
    if reader2id.get(line.strip()) != None:
        reader = reader2id[line.strip()]
        readed = rnn_test_data[reader] if rnn_test_data.get(reader)!=None else [0,0,0,0,0,0,0]
    else:
        reader = reader2id['unk']
        readed = [0,0,0,0,0,0,0]
        a += 1
    test_data.append(readed)

np.save('/data/private/Arena/prepro_results/rnn_test_data.npy', np.array(test_data))
print(a, np.array(test_data).shape)

9 (3000, 7)


In [70]:
# pred_dev_file = '/data/private/Arena/datasets/predict/dev.users'
# pred_dev_data = open(pred_dev_file, 'r').readlines()

# #reader2id = np.load('/data/private/Arena/prepro_results/reader2id.npy', allow_pickle=True).item()
# #reader2items = np.load('/data/private/Arena/prepro_results/reader2item.npy', allow_pickle=True).item()
# dev_mask = np.ones((len(pred_dev_data), 643105))
# for i, line in enumerate(pred_dev_data):
#     try:
#         readed = reader2items[reader2id[line.strip()]]
#         readed = list(set(np.array(readed)[:,0].astype(np.int32).tolist()))
#     except:
#         continue
#     dev_mask[i,readed] = 0
# dev_mask[:,0] = 0
    
# np.save('/data/private/Arena/prepro_results/dev_mask.npy', dev_mask)


Train Valid Test data and Mask (from 4 to 2 : window size = 6)

In [127]:
rnn_train_data = []
rnn_valid_data = []
rnn_test_data = {}
window_size = 6


for reader, items_list in reader2item.items():
    if len(items_list)==0:
        continue
    items_array = np.array(items_list)
    items, read_ts = items_array[:,0].tolist(), items_array[:,1].tolist()

    if len(items) < window_size:
        items = [item_dict['unk']] * (window_size-len(items)) + items
        read_ts = [0] * (window_size-len(read_ts)) + read_ts
    
    rnn_data = []
    for i in range(len(items)-window_size+1):
        rnn_data.append([reader, read_ts[i+window_size-2], read_ts[i+window_size-1]] + items[i:i+window_size])
        
    rnn_test_data[reader] = rnn_data[-1]
    if len(items_list) < 4:
        continue
        
    if len(items) > window_size+4:
        rnn_train_data += rnn_data[:int(len(rnn_data)*0.9)]
        rnn_valid_data += rnn_data[int(len(rnn_data)*0.9):]
    else:
        rnn_train_data += rnn_data

np.save('/data/private/Arena/prepro_results/rnn_train_data2.npy', rnn_train_data)
np.save('/data/private/Arena/prepro_results/rnn_valid_data2.npy', rnn_valid_data)

pred_dev_file = '/data/private/Arena/datasets/predict/dev.users'
pred_dev_data = open(pred_dev_file, 'r').readlines()
test_data = []
a = 0
for line in pred_dev_data:
    if reader2id.get(line.strip()) != None:
        reader = reader2id[line.strip()]
        readed = rnn_test_data[reader] if rnn_test_data.get(reader)!=None else [0] * (window_size+3)
    else:
        reader = reader2id['unk']
        readed = [0]* (window_size+3)
        a += 1
    test_data.append(readed)

np.save('/data/private/Arena/prepro_results/rnn_test_data2.npy', np.array(test_data))

In [126]:
len(rnn_train_data), len(rnn_valid_data)

(1076035, 126306)

# Pretrain Data Build

In [117]:
num_readers = 310759
num_writers = 19066
data_list = []

for reader_idx in tqdm(range(1, num_readers)):
    follow_list = reader2elem[reader_idx]
    if np.nonzero(follow_list)[0].shape[0] < 3:
        continue

    follow_list = np.array(follow_list)[np.nonzero(follow_list)[0]]
    np.random.shuffle(follow_list)
    negs_list = np.array(
                random.sample(list(set(range(num_writers))-set(follow_list)),
                              len(follow_list)-1))
    for i in range(len(follow_list)-1):
        data_list.append([reader_idx, follow_list[i], follow_list[i+1], 1])
        data_list.append([reader_idx, follow_list[i], negs_list[i], 0])
        
data_array = np.array(data_list)
np.save('/data/private/Arena/prepro_results/pretrain_data.npy', data_array)

In [235]:
num_items = 643105
num_magazines = 28130

magazine2keyword = {0:[]}
for data_ in tqdm(metadata_list):
    magazine_id = magazine2id.get(data_['magazine_id'])
    if magazine_id == None:
        continue
    keyword_list = data_['keyword_list']
    
    if magazine2keyword.get(magazine_id) != None:
        magazine2keyword[magazine_id] = list(set(magazine2keyword[magazine_id]+keyword_list))
    else:
        magazine2keyword[magazine_id] = keyword_list

In [214]:
magazine2keyword[0]

In [339]:
ts_min = 1430409600 # 20150501000000 GMT+8
ts_list = []
for elem in list(item2elem.items()):
    ts_list.append((elem[1][6]-ts_min)/8640000)
np.save('/data/private/Arena/prepro_results/ts_array.npy', np.array(ts_list))

In [92]:
print(total)

153


In [91]:
lines = open('./recommend.txt').readlines()
total = 0
for line in lines:
    for item in line.strip().split(' ')[1:]:
        if item in unk_item_name:
            total += 1
            break

In [77]:
unk_item_name = item_list[unk_items]
print(unk_item_name)


['@hukho_247' '@swkyung0221_3' '@iamlove_265' ... '@kindoublej_261'
 '@dreamactivist_2' '@yumileewyky_314']
